$\newcommand{\calf}{{\cal F}}
\newcommand{\dnu}{d \nu}
\newcommand{\mf}{{\bf F}}
\newcommand{\vu}{{\bf u}}
\newcommand{\ve}{{\bf e}}
\newcommand{\mg}{{\bf G}}
\newcommand{\ml}{{\bf L}}
\newcommand{\mg}{{\bf G}}
\newcommand{\mi}{{\bf I}}
\newcommand{\diag}{\mbox{diag}}
\newcommand{\begeq}{{\begin{equation}}}
\newcommand{\endeq}{{\end{equation}}}
$


# Newton's Method in Multiple Precision: C. T. Kelley

This notebook documents the results in 
 <cite data-cite="ctk:sirev20"><a href="newtonmp.html#ctk:sirev20">(Kel20)</cite><br>
    
C. T. Kelley, ***Newton's Method in Mixed Precision***, 2020    

As an example we will solve the Chandrasekhar H-equation <cite data-cite="chand"><a href="newtonmp.html#chand">(Cha60)</cite>. This equation, which we describe in detail in the Example section, has a fast $O(N \log(N))$ function evaluation, a Jacobian evaluation that is $O(N^2)$ work analytically and $O(N^2 \log(N))$ with a finite difference. This means that most of the work, if you do things right, is in the LU factorization of the Jacobian. 
    
The difference between double, single, and half precision will be clear in the results from the examples.
  

## Contents

- [Example: The Chandrasekhar H-Equation](#The-Chandrasekhar-H-Equation)

- [Setting up the notebook](#Setting-up): Install the application.

- [First run of the solver](#Running-the-solver)

- [How to use the solver](#KNL)

- [The results in the paper](#The-results-in-the-paper)

## The Chandrasekhar H-Equation




The example is the mid-point rule discretization of the Chandrasekhar H-equation <cite data-cite="chand"><a href="newtonmp.html#chand">(Cha60)</cite>.
    
    
 
   \begin{equation}
    {\calf}(H)(\mu) = H(\mu) -
\left(
1 - \frac{c}{2} \int_0^1 \frac{\mu H(\mu)}{\mu + \nu} \dnu
\right)^{-1} = 0.
    \end{equation}
    
    
The nonlinear operator $\calf$ is defined on $C[0,1]$, the space of
continuous functions on $[0,1]$.  

The equation has a well-understood dependence on the parameter $c$
<cite data-cite="twm68"><a href="newtonmp.html#twm68">(Mul68)</cite>,
    <cite data-cite="ctk:n1"><a href="newtonmp.html#ctk:n1">(DK80)</cite>.
The equation has unique solutions at $c=0$
and $c=1$ and two solutions for $0 < c < 1$. There is a simple fold
singularity 
        <cite data-cite="herb"><a href="newtonmp.html#herb">(Kel87)</cite>
        at $c=1$. Only one 
        <cite data-cite="chand"><a href="newtonmp.html#chand">(Cha60)</cite>,
        <cite data-cite="busb"><a href="newtonmp.html#busb">(Bus60)</cite>
            of the two solutions for $0 < c < 1$ is of physical interest
and that is the one easiest to find numerically. One must do
a continuation computation to find the other one.

The structure of the singularity is preserved if one discretizes
the integral with any rule that integrates constants exactly. For
the purposes of this paper the composite midpoint rule will suffice.
The $N$-point composite midpoint rule is
\begin{equation}
\int_0^1 f(\nu) \dnu \approx \frac{1}{N} \sum_{j=1}^N f(\nu_j)
\end{equation}
where $\nu_j = (j - 1/2)/N$ for $1 \le j \le N$. This rule is
second-order accurate for sufficiently smooth functions $f$. The
solution of the integral equation is, however, not smooth enough. $H'(\mu)$
has a logarithmic singularity at $\mu=0$.

The discrete problem is

\begin{equation}
\mf(\vu)_i \equiv
u_i - \left(
1  - \frac{c}{2N} \sum_{j=1}^N \frac{u_j \mu_i}{\mu_j + \mu_i}
\right)^{-1}
=0.
\end{equation}

One can simplify the approximate integral operator in \eqnok{hmid}
and expose some useful structure. Since


\begin{equation}
\frac{c}{2N} \sum_{j=1}^N \frac{u_j \mu_i}{\mu_j + \mu_i}
= \frac{c (i - 1/2) }{2N} \sum_{j=1}^N \frac{u_j}{i+j -1}.
\end{equation}


hence the approximate integral operator is
the product of a diagonal matrix and a Hankel matrix and
one can use an FFT to evaluate that operator with $O(N \log(N))$
work 
<cite data-cite="golub"><a href="newtonmp.html#golub">(GV96)</cite>.
    
We can express the approximation of the integral operator in matrix
form
\begin{equation}
\ml(\vu)_i = \frac{c (i - 1/2) }{2N} \sum_{j=1}^N \frac{u_j}{i+j -1}
\end{equation}
and compute the Jacobian analytically as
\begin{equation}
\mf'(\vu) = \mi - \diag(\mg(\vu))^2 \ml
\end{equation}
where
\begin{equation}
\mg(\vu)_i = \left(
1  - \frac{c}{2N} \sum_{j=1}^N \frac{u_j \mu_i}{\mu_j + \mu_i}
\right)^{-1}.
\end{equation}
Hence the data for the Jacobian is already available after
one computes $\mf(\vu) = \vu - \mg(\vu)$ and the Jacobian can
be computed with $O(N^2)$ work.
We do that in this example and therefore the only $O(N^3)$
part of the solve is the matrix factorization.

One could also approximate the Jacobian with forward differences.
In this case one approximates the $j$th column $\mf'(\vu)_j$
of the Jacobian with
\begin{equation}
\frac{\mf(\vu + h {\tilde \ve}_j) - \mf(\vu)}{h}
\end{equation}
where ${\tilde \ve}_j$ is a unit vector in the $j$th coordinate
direction and $h$ is a suitable difference increment. If one computes
$\mf$ in double precision with unit roundoff $u_d$, then
$h =O(\| \vu \| \sqrt{u_d})$ is a reasonable choice 
<cite data-cite="ctk:roots"><a href="newtonmp.html#ctk:roots">(Kel95)</cite>.
Then
the error in the Jacobian is $O(\sqrt{u_d}) = O(u_s)$ where $u_s$ is
unit roundoff in single precision. The cost of a finite difference Jacobian
in this example is $O(N^2 \log(N))$ work.

The analysis in  <cite data-cite="ctk:sirev19"><a href="newtonmp.html#ctk:sirev19">(Kel19)</cite> suggests that there
is no significant difference in the nonlinear iteration
from either the choice of analytic or finite difference Jacobians
or the choice of single or double precision for the linear solver. This notebook has the data used in that paper
to support that assertion. You will be able to duplicate the results and play with the codes.
    
Half precision is another story and we have those codes for you, too.
    


## Setting up



You need to install these packages:

- PyPlot
- LinearAlgebra
- JLD2
- Printf
- FFTW
- IJulia (You must have done this already or you would not be looking at this notebook.)
- AbstractFFTs

The directory is a Julia project. So all you should need to do to get going is to

1. Put the directory in your LOAD_PATH. The way to do this is to type
```Julia
push!(LOAD_PATH,"/Users/yourid/whereyouputit/MPResults2019")
```
at the Julia prompt in the REPL or in a notebook code windown.

2. Now load the modules with
```Julia
using MPResults2019
```

3. Then you can do a simple solve and test that you did it right by typing
```Julia
hout=heqtest()
```
which I will do now. Make sure __MPResults__ is in your LOAD_PATH! If you forget to do the push! command, strange things may happen.

Make absolutely sure that you are in the MPResults directory. Then ...


In [16]:

cd("/Users/ctk/Dropbox/Julia/dev/MPResults")
pwd()
MPRdir=pwd()
push!(LOAD_PATH,MPRdir)
using MPResults


ArgumentError: ArgumentError: Package MPResults [aecca6c3-dca7-5716-8494-c57fe5e8d3bf] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


## Running the solver

The codes solve the H-equation and plot/tabulate the results in various ways. __heqtest__ prints on column of the tables in Chandrasekhar's book. It calls __knl.jl__ . I've shown you the output from the solver but that is not important for now. You get the details on the solver in the next section.

In [5]:
heqtest()

UndefVarError: UndefVarError: heqtest not defined

heqtest.jl, calls the solver and harvests some iteration statistics. The two columns of numbers are the reults from <cite data-cite="chand"><a href="newtonmp.html#chand">(Cha60)</cite> (page 125). The iteration statistics are from KNL, the solver.

##  KNL



The solver is _knl.jl_ version .01. Keep in mind that nothing with a version number with a negative exponent field is likely to be very good. knl.jl is included when you run the MPResults2019.jl module, which you do automatically when you type __using MPResults2019__


knl.jl is a simple implemention of Newton's method (see 
<cite data-cite="ctk:roots"><a href="newtonmp.html#ctk:roots">(Kel95)</cite> and
<cite data-cite="ctk:newton"><a href="newtonmp.html#ctk:newton">(Kel03)</cite> )
using an LU factorization of the Jacobian to compute the Newton step with no line search or globalization. The code evaluates and factors the Jacobian at every nonlinear iteration.


## Using knl.jl ##

At the level of this notebook, it's pretty simple. Remember that Julia hates to allocate mememory. So your function and Jacobian evaluation routines should expect the calling function to **preallocate** the storage for both the function and Jacobian. Your functions will then use __.=__ to put the function and Jacobian where they are supposed to be.

It's worthwhile to look at the help screen. 

It's worth asking for help with knl ....

In [6]:
?knl

search: installkernel

Couldn't find knl
Perhaps you meant kill, In, ans, nb, nnz, Inf, Val, all, any, in, inv or one


No documentation found.

Binding `knl` does not exist.


## How knl.jl controls the precision of the Jacobian##

You can control the precision of the Jacobian by simply allocating FPS in your favorite precision. So if I have a problem with N=256 unknows I will decalare FP as zeros(N,1) and may delcare FPS as zeros(N,N) or Float32.(zeros(N,N)).

Note the __.__ between Float32 and the paren. This, as is standard Julia practice, applies the conversion to Float32 to everyelement in the array. If you forget the __.__ Julia will complain.

# The results in the paper



The paper has plots for double, single, and half precsion computations for c=.5, .99, and 1.0. The half precision results take a very long time to get. On my computer (2019 iMac; 8 cores, 64GB of memoroy) the half precision compute time was over two weeks. Kids, don't try this at home.

The data for the paper are in the cleverly named directory __Data_From_Paper__

__cd to the directory MPResults__ and __from that directory__ run

```Julia
data_harvest("Data_From_Paper/MP_Data_") 
```

at the julia prompt you will generate all the tables and plots in the paper.

If you have the time and patience you can also generate the data with data_populate.jl. This create three directories named Mixed_Precision_c=? and you can see for yourself. Look at that file to see opportunities to edit the time-cosumng jobs out. If you eliminate the half precision work and the larger dimensions, the code will run in a short time. 

Here is a simple example of using data_populate and the plotter code plot_knl.jl. I'm only using the 1024, 2048 and 4096 point grids. The plot in the paper uses more levels. This is part of Figure 1 in the paper.

Look at the source to __data_populate.jl__ and __plotknl.jl__ and you'll see how I did this. These codes only mange files, plots, and tables. There is nothing really exciting here. You don't need to know much Julia to understand this, but you do need to know something and I can't help with that.

To begin with, I will create a directory called Data_Test to put all this stuff. I will cd to that directory.

In [7]:
Home4mp="Data_Test"
try (mkdir(Home4mp))
catch
end
cd(Home4mp)
pwd()

"/Users/ctk/Dropbox/Julia/dev/MPResults/Data_Test/Data_Test"

Now I'll run **data_populate** to create a subdirectory with the data. cd to that directory and run __plotknl__. That's how I created the plots in the paper with all values of *c* and all the problem sizes. The half precision computation took two weeks.

In [10]:
using PyPlot
data_populate(.5;half="no",level=3)

UndefVarError: UndefVarError: data_populate not defined

In [9]:
cd("Mixed_Precision_c=5")
figtitle="Figure 1"
plotknl("no",.5,10,3;bigtitle=figtitle)
cd("..")


Base.IOError: IOError: chdir Mixed_Precision_c=5: no such file or directory (ENOENT)

__data_populate__ will create directories for your data if they are not already there. Run __plot_knl__ from one of those directories and it makes the plots. As you can see from the one above, it's hard to tell the difference between double and single precision linear algebra and analytic or finite difference Jacobians. You knew that.